In [34]:
%pip install -r ./../requirement.txt

Note: you may need to restart the kernel to use updated packages.


In [35]:
# Bibliotecas necesarias
import pandas as pd
import numpy as np
import requests
import os

# Reportes de folios

In [42]:
servidores = {
    "centro" : "26.61.16.123",
    "araucarias" : "26.217.212.35",
    "cedis" : "26.207.163.198"
}

In [43]:
# Definir los parametros de la API
try:
    parametros = {
        "fecha": "2024-09-28",
        "servidor" : servidores["centro"]
    }
    api = f'http://{parametros["servidor"]}:8000'
except Exception as e:
    print("El servidor no se encuentra disponible")

In [44]:
# Hacer la peticion a la API
try:
    ventas = requests.get(f'{api}/mantenimiento/ventas', json=parametros)
    ventas = ventas.json()
except requests.exceptions.RequestException as e:
    print("Error: El endpoint no esta disponible, intente mas tarde o contacte con el administrador")
    exit()

In [45]:
# Crear el Marco de Datos para las ventas
df_ventas = pd.DataFrame(ventas)

In [46]:
# Cambiar las celdas efectivo, tarjeta y total a float
df_ventas["efectivo"] = df_ventas["efectivo"].astype(float)
df_ventas["tarjeta"] = df_ventas["tarjeta"].astype(float)
df_ventas["total"] = df_ventas["total"].astype(float)

In [52]:
# Folios: efectivo >= a 120 & tarjeta == 0 & facturado == false
folios_mantenimiento = df_ventas[(df_ventas["efectivo"] >= 120) & (df_ventas["tarjeta"] == 0) & (df_ventas["facturado"] == False)]
folios_mantenimiento.to_csv(f'Folios_Para_Procesar_{parametros["fecha"]}.csv', index=False)

# Tamaño / Numero de folios disponibles para aplicar un mantenimiento
num_folios = np.array(folios_mantenimiento["folio"]).size

In [54]:
# Folios: facturado == true & efectivo > 0 & tarjeta == 0
folios_facturados_efectivo = df_ventas[(df_ventas["facturado"] == True) & (df_ventas["efectivo"] > 0) & (df_ventas["tarjeta"] == 0)]
folios_facturados_efectivo.to_csv(f'Folios_Facturados_Efectivo_{parametros["fecha"]}.csv', index=False)

# Numero de folios facturados con efectivo
num_folios_facturados_efectivo = np.array(folios_facturados_efectivo["folio"]).size